In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import json
import os
from datetime import datetime

In [ ]:
# Load dataset
df = pd.read_csv('finance_data.csv')
print(f"Dataset loaded: {df.shape[0]} records")

In [ ]:
# Encode categorical columns
le_occ = LabelEncoder()
le_city = LabelEncoder()
df['Occupation_encoded'] = le_occ.fit_transform(df['Occupation'])
df['City_Tier_encoded'] = le_city.fit_transform(df['City_Tier'])

In [ ]:
# Create expense columns
expense_cols = ['Rent','Loan_Repayment','Insurance','Groceries','Transport','Eating_Out',
                'Entertainment','Utilities','Healthcare','Education','Miscellaneous']
df['Total_Expenses'] = df[expense_cols].sum(axis=1)

In [ ]:
# Train savings prediction model
features = [
    'Income', 'Age', 'Dependents', 'Occupation_encoded', 'City_Tier_encoded',
    'Total_Expenses', 'Desired_Savings_Percentage', 'Disposable_Income'
]
X = df[features]
y = df['Desired_Savings']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_savings = GradientBoostingRegressor(random_state=42)
model_savings.fit(X_train, y_train)
print("Savings model trained")

In [ ]:
# Train overspending risk model
# Overspending = when expenses + desired savings exceed income
df['Required_Total'] = df['Total_Expenses'] + df['Desired_Savings']
df['Overspend'] = (df['Required_Total'] > df['Income']).astype(int)
X_risk = df[['Income', 'Age', 'Dependents', 'Occupation_encoded', 'City_Tier_encoded', 'Total_Expenses', 'Desired_Savings_Percentage']]
y_risk = df['Overspend']
clf_risk = RandomForestClassifier(random_state=42)
clf_risk.fit(X_risk, y_risk)
print("Risk model trained")

In [ ]:
# User prediction function
_default_payload = {
    "Income": 60000.0,
    "Age": 30.0,
    "Dependents": 1.0,
    "Occupation_encoded": 1.0,
    "City_Tier_encoded": 1.0,
    "Total_Expenses": 7800.0,
    "Desired_Savings_Percentage": 15.0,
    "Disposable_Income": 35000.0,
    "Rent": 0.0,
    "Loan_Repayment": 0.0,
    "Insurance": 0.0,
    "Groceries": 0.0,
    "Transport": 0.0,
    "Eating_Out": 0.0,
    "Entertainment": 0.0,
    "Utilities": 0.0,
    "Healthcare": 0.0,
    "Education": 0.0,
    "Miscellaneous": 0.0,
}
_default_labels = {
    "Occupation": "Salaried",
    "City_Tier": "Tier 2",
}

def _normalize_payload(payload: dict) -> dict:
    values = _default_payload.copy()
    for key in values:
        try:
            values[key] = float(payload.get(key, values[key]))
        except (TypeError, ValueError):
            values[key] = values[key]
    return values

def generate_chart_data(user_values):
    """Generate updated chart data based on user input and dataset"""
    # Sample 10 records from dataset for scatter plot
    sample_df = df.sample(n=min(10, len(df)), random_state=42)
    
    # Scatter: Income vs Total Expenses
    scatter_data = {
        "income": sample_df['Income'].tolist(),
        "totalExpenses": sample_df['Total_Expenses'].tolist(),
        "cityTier": sample_df['City_Tier'].tolist(),
        "savingsPct": sample_df['Desired_Savings_Percentage'].tolist()
    }
    
    # Pie: Average expense breakdown
    pie_data = {
        "labels": expense_cols,
        "values": [round(df[col].mean(), 2) for col in expense_cols]
    }
    
    # Bar: Average expenses by occupation
    occupation_stats = df.groupby('Occupation')['Total_Expenses'].mean().sort_values(ascending=False)
    bar_data = {
        "labels": occupation_stats.index.tolist(),
        "values": [round(v, 2) for v in occupation_stats.values.tolist()]
    }
    
    # Projection: 12-month savings based on user's predicted savings
    predicted_monthly = max(user_values.get('predicted_savings', 0), 0)  # Don't project negative savings
    interest_rate = 0.06  # 6% annual
    projection_values = []
    for month in range(1, 13):
        if predicted_monthly > 0:
            projected = predicted_monthly * month * (1 + interest_rate/12)
        else:
            projected = 0
        projection_values.append(round(projected, 2))
    
    projection_data = {
        "months": list(range(1, 13)),
        "values": projection_values
    }
    
    # Heatmap: Correlation matrix
    corr_cols = ['Income', 'Total_Expenses', 'Desired_Savings_Percentage', 'Disposable_Income', 'Dependents']
    corr_matrix = df[corr_cols].corr()
    heatmap_data = {
        "labels": corr_cols,
        "matrix": corr_matrix.values.tolist()
    }
    
    return {
        "scatter": scatter_data,
        "pie": pie_data,
        "bar": bar_data,
        "projection": projection_data,
        "heatmap": heatmap_data
    }

def analyze_expense_categories(user_values, income):
    """Analyze each expense category and provide detailed breakdown with savings opportunities"""
    user_expenses = {
        'Rent': user_values.get('Rent', 0),
        'Loan_Repayment': user_values.get('Loan_Repayment', 0),
        'Insurance': user_values.get('Insurance', 0),
        'Groceries': user_values.get('Groceries', 0),
        'Transport': user_values.get('Transport', 0),
        'Eating_Out': user_values.get('Eating_Out', 0),
        'Entertainment': user_values.get('Entertainment', 0),
        'Utilities': user_values.get('Utilities', 0),
        'Healthcare': user_values.get('Healthcare', 0),
        'Education': user_values.get('Education', 0),
        'Miscellaneous': user_values.get('Miscellaneous', 0),
    }
    
    # Calculate dataset averages and recommended percentages
    avg_expenses = {col: df[col].mean() for col in expense_cols}
    
    # Industry-standard recommended percentages of income
    recommended_pct = {
        'Rent': 30.0,
        'Loan_Repayment': 10.0,
        'Insurance': 5.0,
        'Groceries': 10.0,
        'Transport': 10.0,
        'Eating_Out': 5.0,
        'Entertainment': 5.0,
        'Utilities': 5.0,
        'Healthcare': 5.0,
        'Education': 5.0,
        'Miscellaneous': 5.0,
    }
    
    category_analysis = []
    total_potential_savings = 0
    
    for category in expense_cols:
        amount = user_expenses.get(category, 0)
        if amount == 0:
            continue
            
        avg_amount = avg_expenses.get(category, 0)
        recommended_amount = income * (recommended_pct.get(category, 5) / 100)
        user_pct = (amount / income) * 100 if income > 0 else 0
        
        status = "\u2713 Good"
        potential_saving = 0
        advice = []
        
        # Determine status and potential savings
        if amount > recommended_amount * 1.3:  # 30% above recommended
            status = "\U0001F534 High"
            potential_saving = amount - recommended_amount
            advice.append(f"Reduce to recommended \u20B9{recommended_amount:,.0f} ({recommended_pct.get(category, 5):.0f}% of income)")
        elif amount > recommended_amount * 1.1:  # 10% above recommended
            status = "\U0001F7E1 Moderate"
            potential_saving = amount - recommended_amount
            advice.append(f"Consider reducing to \u20B9{recommended_amount:,.0f}")
        elif amount > avg_amount * 1.2:  # 20% above dataset average
            status = "\U0001F7E1 Above Average"
            potential_saving = amount - avg_amount
            advice.append(f"Dataset average is \u20B9{avg_amount:,.0f}")
        else:
            advice.append("Spending is within healthy range")
        
        # Category-specific tips
        if category == 'Rent' and amount > income * 0.30:
            advice.append("\U0001F4A1 Consider relocating or finding roommates")
        elif category == 'Eating_Out' and amount > income * 0.05:
            advice.append("\U0001F4A1 Meal prep at home can save 50-70%")
        elif category == 'Entertainment' and amount > income * 0.05:
            advice.append("\U0001F4A1 Explore free/low-cost activities")
        elif category == 'Transport' and amount > income * 0.10:
            advice.append("\U0001F4A1 Use public transport or carpool")
        elif category == 'Groceries' and amount > income * 0.12:
            advice.append("\U0001F4A1 Use shopping lists, buy in bulk, avoid waste")
        elif category == 'Utilities' and amount > income * 0.05:
            advice.append("\U0001F4A1 Reduce AC/heating usage, switch to LED lights")
        elif category == 'Miscellaneous' and amount > income * 0.05:
            advice.append("\U0001F4A1 Track and eliminate impulse purchases")
        
        total_potential_savings += potential_saving
        
        category_analysis.append({
            "category": category.replace('_', ' '),
            "current_amount": round(amount, 2),
            "current_percentage": round(user_pct, 2),
            "recommended_amount": round(recommended_amount, 2),
            "recommended_percentage": recommended_pct.get(category, 5),
            "dataset_average": round(avg_amount, 2),
            "status": status,
            "potential_saving": round(potential_saving, 2),
            "advice": advice
        })
    
    # Sort by potential savings (highest first)
    category_analysis.sort(key=lambda x: x['potential_saving'], reverse=True)
    
    return {
        "categories": category_analysis,
        "total_potential_savings": round(total_potential_savings, 2),
        "summary": f"Total potential savings across all categories: \u20B9{total_potential_savings:,.2f}/month"
    }

def generate_recommendations(user_values, predicted_savings, overspend_prob, category_breakdown):
    """Generate actionable saving recommendations based on expense analysis"""
    recommendations = []
    
    income = user_values.get('Income', 0)
    expenses = user_values.get('Total_Expenses', 0)
    savings_pct = user_values.get('Desired_Savings_Percentage', 0)
    desired_savings_amount = income * (savings_pct / 100)
    available_after_expenses = income - expenses
    
    # Primary recommendation based on shortfall/surplus
    if available_after_expenses < desired_savings_amount:
        shortfall = desired_savings_amount - available_after_expenses
        recommendations.append(f"\U0001F3AF Target: Reduce expenses by \u20B9{shortfall:,.0f}/month to meet your {savings_pct}% savings goal.")
        
        # Highlight top savings opportunities from category breakdown
        high_priority = [cat for cat in category_breakdown['categories'] if cat['potential_saving'] > 0][:3]
        
        if high_priority:
            recommendations.append("\U0001F525 Top 3 savings opportunities:")
            cumulative_savings = 0
            for cat in high_priority:
                cumulative_savings += cat['potential_saving']
                recommendations.append(f"   \u2022 {cat['category']}: \u20B9{cat['current_amount']:,.0f} \u2192 \u20B9{cat['recommended_amount']:,.0f} (save \u20B9{cat['potential_saving']:,.0f}/month)")
            
            if cumulative_savings >= shortfall:
                recommendations.append(f"   \u2713 Implementing these changes would cover your \u20B9{shortfall:,.0f} shortfall!")
            else:
                remaining = shortfall - cumulative_savings
                recommendations.append(f"   \u26A0\uFE0F Additional \u20B9{remaining:,.0f}/month reduction needed")
        
        # Quick wins
        recommendations.append("\U0001F4CB Quick wins (easiest to implement):")
        quick_wins = []
        for cat in category_breakdown['categories']:
            if cat['category'] in ['Eating Out', 'Entertainment', 'Miscellaneous'] and cat['potential_saving'] > 0:
                quick_wins.append(f"   \u2022 {cat['category']}: {cat['advice'][0]}")
        
        if quick_wins:
            recommendations.extend(quick_wins[:3])
        else:
            recommendations.append("   \u2022 Review all discretionary spending categories")
    
    else:
        # Surplus scenario
        surplus = available_after_expenses - desired_savings_amount
        recommendations.append(f"\u2713 Excellent! You have \u20B9{surplus:,.0f}/month surplus after meeting your {savings_pct}% savings goal.")
        recommendations.append("\U0001F4B0 Smart moves for your surplus:")
        recommendations.append(f"   \u2022 Build emergency fund: Target 6-months expenses (\u20B9{expenses * 6:,.0f})")
        recommendations.append(f"   \u2022 Invest \u20B9{surplus * 0.6:,.0f} in SIP/Mutual Funds for long-term growth")
        recommendations.append(f"   \u2022 Keep \u20B9{surplus * 0.4:,.0f} as buffer for unexpected expenses")
        
        # Still show optimization opportunities
        optimizations = [cat for cat in category_breakdown['categories'] if cat['potential_saving'] > 100]
        if optimizations:
            recommendations.append("\U0001F3AF Further optimization opportunities:")
            for cat in optimizations[:2]:
                recommendations.append(f"   \u2022 {cat['category']}: {cat['advice'][0]}")
    
    # Risk-based recommendations
    if overspend_prob and overspend_prob > 0.5:
        recommendations.append("\u26A0\uFE0F Risk mitigation strategies:")
        recommendations.append("   \u2022 Set up automatic savings transfer on payday")
        recommendations.append("   \u2022 Use separate accounts for fixed vs discretionary expenses")
        recommendations.append("   \u2022 Set spending alerts for high-risk categories")
    
    # Category-specific deep dives
    if category_breakdown['total_potential_savings'] > 0:
        recommendations.append(f"\U0001F4CA Total optimization potential: \u20B9{category_breakdown['total_potential_savings']:,.2f}/month (\u20B9{category_breakdown['total_potential_savings'] * 12:,.2f}/year)")
    
    return recommendations

def generate_insights(user_values, predicted_savings, overspend_prob):
    """Generate personalized insights based on user data and predictions"""
    insights = []
    
    income = user_values.get('Income', 0)
    expenses = user_values.get('Total_Expenses', 0)
    savings_pct = user_values.get('Desired_Savings_Percentage', 0)
    desired_savings_amount = income * (savings_pct / 100)
    available_after_expenses = income - expenses
    
    # Insight 1: Expense ratio and reality check
    if income > 0:
        expense_ratio = (expenses / income) * 100
        insights.append(f"Your expenses are {expense_ratio:.1f}% of your income (\u20B9{expenses:,.0f} / \u20B9{income:,.0f}).")
        
        # Reality check: Can they actually save?
        if available_after_expenses <= 0:
            insights.append(f"\u26A0\uFE0F Critical: Expenses equal or exceed income. No funds available for savings.")
        elif available_after_expenses < desired_savings_amount:
            shortfall = desired_savings_amount - available_after_expenses
            insights.append(f"\u26A0\uFE0F Gap: Only \u20B9{available_after_expenses:,.0f} available, but \u20B9{desired_savings_amount:,.0f} needed for {savings_pct}% savings goal (shortfall: \u20B9{shortfall:,.0f}).")
        else:
            surplus = available_after_expenses - desired_savings_amount
            insights.append(f"\u2713 Healthy surplus: \u20B9{available_after_expenses:,.0f} available after expenses. Your {savings_pct}% goal (\u20B9{desired_savings_amount:,.0f}) leaves \u20B9{surplus:,.0f} buffer.")
    
    # Insight 2: Savings rate assessment - compare available vs target
    if available_after_expenses > 0:
        actual_savings_pct = (available_after_expenses / income) * 100
        
        # Compare against user's goal first
        if actual_savings_pct >= savings_pct:
            # Can meet or exceed goal
            if actual_savings_pct >= savings_pct * 1.1:  # 10% above goal
                insights.append(f"\u2713 Excellent: {actual_savings_pct:.1f}% available, exceeding your {savings_pct}% goal!")
            else:
                insights.append(f"\u2713 Good: {actual_savings_pct:.1f}% available, meeting your {savings_pct}% savings goal.")
        else:
            # Cannot meet goal
            gap_pct = savings_pct - actual_savings_pct
            insights.append(f"\u26A0\uFE0F Below Target: Only {actual_savings_pct:.1f}% available, falling short of your {savings_pct}% goal by {gap_pct:.1f}%.")
    elif predicted_savings < 0:
        insights.append(f"\u274C Deficit situation: Your {savings_pct}% target is unachievable with current expenses. Reduce expenses or adjust savings goal.")
    
    # Insight 3: Overspending risk
    if overspend_prob is not None:
        if overspend_prob < 0.3:
            if available_after_expenses >= desired_savings_amount:
                insights.append(f"\u2713 Low overspending risk ({overspend_prob*100:.1f}%). Income comfortably covers expenses + savings goals.")
            else:
                insights.append(f"\u26A0\uFE0F Low overspending risk ({overspend_prob*100:.1f}%), but savings target exceeds available funds.")
        elif overspend_prob < 0.6:
            insights.append(f"\u26A0\uFE0F Moderate risk ({overspend_prob*100:.1f}%). Expenses + savings goals strain your income.")
        else:
            insights.append(f"\u274C High risk ({overspend_prob*100:.1f}%). Expenses + savings goals exceed income. Budget adjustment critical.")
    
    # Insight 4: Predicted savings context - clearly show what's achievable vs desired
    if predicted_savings > 0:
        monthly_savings = predicted_savings
        annual_savings = monthly_savings * 12
        
        # Check if there's a shortfall
        if available_after_expenses < desired_savings_amount:
            # User CANNOT meet their goal
            shortfall = desired_savings_amount - available_after_expenses
            insights.append(f"\U0001F4CA Achievable: \u20B9{monthly_savings:,.2f}/month (\u20B9{annual_savings:,.2f}/year). Shortfall: \u20B9{shortfall:,.2f}/month (\u20B9{shortfall * 12:,.2f}/year) below your {savings_pct}% goal of \u20B9{desired_savings_amount:,.2f}/month.")
        else:
            # User CAN meet or exceed their goal
            surplus = available_after_expenses - desired_savings_amount
            if surplus > 100:  # Meaningful surplus
                insights.append(f"\U0001F4CA Target achieved: \u20B9{monthly_savings:,.2f}/month (\u20B9{annual_savings:,.2f}/year) meets your {savings_pct}% goal. Additional \u20B9{surplus:,.2f}/month available for extra savings or investments.")
            else:
                insights.append(f"\U0001F4CA Target achieved: \u20B9{monthly_savings:,.2f}/month (\u20B9{annual_savings:,.2f}/year) meets your {savings_pct}% savings goal.")
    elif predicted_savings < 0:
        deficit = abs(predicted_savings)
        annual_deficit = deficit * 12
        insights.append(f"\U0001F4CA Monthly shortfall: -\u20B9{deficit:,.2f} (-\u20B9{annual_deficit:,.2f} annually). Immediate action required.")
    else:
        insights.append(f"\U0001F4CA Breakeven: Expenses consume all available income after meeting savings target.")
    
    return insights

def run_user_prediction(payload: dict) -> dict:
    values = _normalize_payload(payload)
    occupation_label = payload.get("Occupation", _default_labels["Occupation"])
    city_label = payload.get("City_Tier", _default_labels["City_Tier"])
    
    # Encode categorical values
    if occupation_label:
        try:
            values["Occupation_encoded"] = float(le_occ.transform([occupation_label])[0])
        except Exception:
            pass
    if city_label:
        try:
            values["City_Tier_encoded"] = float(le_city.transform([city_label])[0])
        except Exception:
            pass

    # Calculate savings mathematically
    income = values.get("Income", 0)
    expenses = values.get("Total_Expenses", 0)
    savings_pct = values.get("Desired_Savings_Percentage", 0)
    
    available_after_expenses = income - expenses
    desired_savings_amount = income * (savings_pct / 100)
    
    # Predicted savings is what's left after expenses, capped by desired target
    if available_after_expenses >= desired_savings_amount:
        predicted = desired_savings_amount
    else:
        predicted = available_after_expenses

    # Predict overspend probability
    overspend_payload = pd.DataFrame([{
        "Income": values["Income"],
        "Age": values["Age"],
        "Dependents": values["Dependents"],
        "Occupation_encoded": values["Occupation_encoded"],
        "City_Tier_encoded": values["City_Tier_encoded"],
        "Total_Expenses": values["Total_Expenses"],
        "Desired_Savings_Percentage": values["Desired_Savings_Percentage"]
    }])
    overspend_prob = None
    try:
        overspend_prob = float(clf_risk.predict_proba(overspend_payload)[0][1])
    except Exception:
        overspend_prob = None

    # Generate detailed category breakdown
    category_breakdown = analyze_expense_categories(values, income)
    
    # Generate chart data, insights, and recommendations
    values['predicted_savings'] = predicted
    chart_data = generate_chart_data(values)
    insights = generate_insights(values, predicted, overspend_prob)
    recommendations = generate_recommendations(values, predicted, overspend_prob, category_breakdown)

    result = {
        "predicted_desired_savings": round(predicted, 2),
        "desired_savings_amount": round(desired_savings_amount, 2),
        "shortfall": round(max(0, desired_savings_amount - available_after_expenses), 2),
        "overspend_probability": overspend_prob,
        "input": values,
        "generated_at": datetime.utcnow().isoformat() + "Z",
        "charts": chart_data,
        "insights": insights,
        "recommendations": recommendations,
        "expense_breakdown": category_breakdown
    }
    
    # Write result to file
    with open("user_prediction.json", "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    
    return result


In [ ]:
# Execute user prediction if payload provided
if os.environ.get("USER_INPUT_PAYLOAD"):
    payload = json.loads(os.environ["USER_INPUT_PAYLOAD"])
    user_result = run_user_prediction(payload)
    print("USER_RESULT", json.dumps(user_result))
else:
    print("No user input payload provided")